## Import the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import f1_score

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Load in your data from kaggle.  
By working in a kaggle kernel, you can access the data directly from the competition, as well as make your submission without downloading your output file

In [ ]:
train = pd.read_csv('../input/climate-change-edsa2020-21/train.csv')
test = pd.read_csv('../input/climate-change-edsa2020-21/test.csv')

In [ ]:
train.sentiment.value_counts()

In [ ]:
train.head()

## Splitting out the X variable from the target

In [ ]:
y = train['sentiment']
X = train['message']

In [ ]:
from nltk.stem import PorterStemmer

# init stemmer
porter_stemmer=PorterStemmer()

def my_cool_preprocessor(text):
    
    text=text.lower() 
    text=re.sub("\\W"," ",text) # remove special chars
    text=re.sub("\\s+(in|the|all|for|and|on)\\s+"," _connector_ ",text) # normalize certain words
    
    # stem words
    words=re.split("\\s+",text)
    stemmed_words=[porter_stemmer.stem(word=word) for word in words]
    return ' '.join(stemmed_words)

In [ ]:
def my_tokenizer(text):
    # create a space between special characters 
    text=re.sub("(\\W)"," \\1 ",text)

    # split based on whitespace
    return re.split("\\s+",text)

## Turning text into something your model can read

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,2),tokenizer=my_tokenizer, min_df=2,max_df=0.80,analyzer='word',smooth_idf=False, preprocessor=my_cool_preprocessor,stop_words="english")
X_vectorized = vectorizer.fit_transform(X)
, 
#stop_words="english",max_df=0.85, preprocessor=my_cool_preprocessor,"all","in","the","is","and"preprocessor=my_cool_preprocessor

In [ ]:
#vectorizer.vocabulary_
#vectorizer.stop_words_

## Splitting the training data into a training and validation set

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X_vectorized,y,test_size=0.30,shuffle=True, random_state=25)

## Training the model and evaluating using the validation set 

In [ ]:
lsvc = LinearSVC()
lsvc.fit(X_train, y_train)
lsvc_pred = lsvc.predict(X_val)

## Checking the performance of our model on the validation set

In [ ]:
f1_score(y_val, lsvc_pred, average="macro")

In [ ]:
from sklearn import metrics

print(metrics.classification_report(y_val, lsvc_pred))

## Getting our test set ready 

In [ ]:
testx = test['message']
test_vect = vectorizer.transform(testx)

## Making predictions on the test set and adding a sentiment column to our original test df

In [ ]:
y_pred = lsvc.predict(test_vect)

In [ ]:
test['sentiment'] = y_pred

In [ ]:
test.head()

## Creating an output csv for submission

In [ ]:
test[['tweetid','sentiment']].to_csv('testsubmission_8.csv', index=False)